# AFSK Demodulator
## Step 2: HDLC Implemention

This is a Pynq portion of the AFSK demodulator project.  This component impements the HDLC subsystem, which includes the NRZI code and CRC calculation code.

At this point we have created the bitstream for "hdlc" and copied the bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

Let's first verify that we can load the module.

In [29]:
from pynq import Overlay, Xlnk
import numpy as np
import pynq.lib.dma

overlay = Overlay('hdlc.bit')
dma = overlay.demodulator.dma
hdlc = overlay.demodulator.hdlc
print(hdlc.register_map)

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  cancel_V = Register(cancel_V=0, RESERVED=0)
}


## Accellerating HDLC Processing

Below is the implementation of the AFSK demodulator in Python.  We are going to remove the Python HDLC code and replace it with new code.

In [31]:
import sys
sys.path.append('../../base')

import numpy as np
from scipy.signal import lfiltic, lfilter, firwin
from scipy.io.wavfile import read
from DigitalPLL import DigitalPLL
from AX25 import AX25
import time

from pynq import Overlay, Xlnk
import numpy as np
import asyncio
import concurrent.futures

xlnk = Xlnk()

block_size = 65536

class fir_filter(object):
    def __init__(self, coeffs):
        self.coeffs = coeffs
        self.zl = lfiltic(self.coeffs, 32768.0, [], [])
    def __call__(self, data):
        result, self.zl = lfilter(self.coeffs, 32768.0, data, -1, self.zl)
        return result

audio_file = read('../../base/TNC_Test_Ver-1.101-26400-1min.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1]
print(len(audio_data))
delay = 12 # ~446us

bpf_coeffs = np.array(firwin(141, [1100.0/(sample_rate/2),2300.0/(sample_rate/2)], width = None,
        pass_zero = False, scale = True, window='hann') * 32768, dtype=int)
lpf_coeffs = np.array(firwin(101, [760.0/(sample_rate/2)], width = None,
        pass_zero = True, scale = True, window='hann') * 32768, dtype=int)

bpf = fir_filter(bpf_coeffs)
lpf = fir_filter(lpf_coeffs)

filter_delay = 64 + 50

hdlc.register_map.cancel_V = 0

async def read_packet():
    count = 0
    with xlnk.cma_array(shape=(4096,), dtype=np.uint8) as in_buffer:
        while True:
            dma.recvchannel.transfer(in_buffer)
            await dma.recvchannel.wait_async()
            in_buffer.invalidate();
            received = dma.recvchannel._mmio.read(dma.recvchannel._offset + 0x28)
            # print("received", received, in_buffer[received - 1])
            if in_buffer[received - 1] & 2:
                print("Done Reading", received, in_buffer[received - 1])
                break
            if not (received > 12 and in_buffer[received - 1] & 1):
                # print("Bad packet", in_buffer[received - 1])
                continue
            else:
                count += 1
            packet = ''.join(str(s, encoding='Latin-1') for s in in_buffer[:received-3])
            try:
                print(count, AX25(packet), received)
                # sys.stdout.write('\r%05d' % count)
            except:
                print("decode error: ", packet, received)

async def write_hdlc(data_, length):
    print("Generating data...")
    data = np.array([(b<<2)|(s<<1)|l for b,s,l in data_], dtype=np.uint8)
    print("Starting IP...")
    hdlc.register_map.CTRL = 0x81 # START|AUTO-RESTART while writing data
    print("Writing data...")
    with xlnk.cma_array(shape=(block_size,), dtype=np.uint8) as out_buffer:
        for i in range(0, length, block_size):
            size = len(data[i:i+block_size])
            if size != block_size: break
            out_buffer[:] = data[i:i+block_size]
            out_buffer.flush();
            dma.sendchannel.transfer(out_buffer)
            await dma.sendchannel.wait_async()
    
    with xlnk.cma_array(shape=(size,), dtype=np.uint8) as out_buffer:
        out_buffer[:] = data[i:i+size]
        dma.sendchannel.transfer(out_buffer)
        await dma.sendchannel.wait_async()

    print("Done writing")
    hdlc.register_map.CTRL = 0x00 # STOP
    hdlc.register_map.cancel_V = 1
    hdlc.register_map.CTRL = 0x01 # START ONE for cancellation

# Band-pass filter the audio data
print("Doing BPF...")
f = bpf(np.append(audio_data, np.zeros(filter_delay, dtype=int)))
# Digitize the data
print("Digitizing audio data...")
print(len(f))
d = np.greater_equal(f, 0)
print(len(d))
# Delay the data
print("Delay...")
a = d[delay:]
# XOR the digitized data with the delayed version
print("Doing Logical XOR...")
x = np.logical_xor(d[:0-delay], a, dtype=int)
# Low-pass filter the PWM signal
print("Doing LPF...")
c = lpf(x-0.5)
# Digitize the tone transistions
print(len(c))
print("Digitizing correlator output...")
dx = np.greater_equal(c, 0.0)
print(len(dx))
# Create the PLL
pll = DigitalPLL(sample_rate, 1200.0)

locked = np.zeros(len(dx), dtype=int)
sample = np.zeros(len(dx), dtype=int)

# Clock recovery
print("Doing clock recovery...", end="")
for i in range(len(dx)):
    sample[i] = pll(dx[i])
    locked[i] = pll.locked()

print("done")

1584000
Doing BPF...
Digitizing audio data...
1584114
1584114
Delay...
Doing Logical XOR...
Doing LPF...
1584102
Digitizing correlator output...
1584102
Doing clock recovery...done


The above code does all of the audio processing and clock recovery.  These are time-consuming operations, especially the clock recovery part.

Now we can process the bitstream through the HDLC module in the FPGA.  After that we can compare the results to the baseline Python implementation.

In [32]:
def run():

    print("Starting HDLC...")
    start_time = time.time()

    # demodulate the audio data
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(asyncio.gather(
            read_packet(),
            write_hdlc(zip(dx,sample,locked), len(dx))
        ))
    finally:
        pass
        # Cannot close event loop in Jupyter.
        # loop.close()
    
    stop_time = time.time()
    sw_exec_time = stop_time - start_time
    print('Total execution time:', sw_exec_time)

run()

Starting HDLC...
Generating data...
Starting IP...
Writing data...
1 WA6YLB-8>APRS,N6EX-11:$ULTW00000000----0000----000086A00001----0000000000000000?? 85
2 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 78
3 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 78
4 N6XQY-8>GPSLJ,RELAY,WIDE2-5:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73? 104
5 WA6YLB>APRX46,WA6YLB-14,W6SCE-5:>081839z wa6ylb@theworks.com? 62
6 KC6HUR-2>S4QVYV,W6SCE-5:'.4&l-/k/]"7q}? 41
7 N6XQY-8>GPSLJ,N6EX-9:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73? 97
8 KC6BLF-12>S4PWYS,N6EX-11:'-U l{(u/]"5\}Lost in the West!? 58
9 K6KMA-2>GPSLK,N6EX-3:$GPRMC,013647,A,3350.076,N,11806.996,W,028.3,180.5,231105,013.5,E*69?? 96
10 AE6GR-14>S4PXYW,WIDE2-5:'._|l tv/]"6[}? 41
11 AE6GR-14>S4PXYW,N6EX-3:'._|l tv/]"6[}? 41
12 AE6MP>SS5PPQ-4,N6EX-9:`.](n->>/"4W} 39
13 AE6MP>SS5PPQ-4,N6EX-3:`.](n->>/"4W} 39
14 AE6MP>SS5PPQ-2,N6EX-11:`.](n

The next cell is here to perform recovery on the Pynq in case there is a problem or we interrupt the cell above.  When we do that, the DMA engine may be in a state that requires reset.  Run this block if DMA was interrupted.

In [19]:
ctrl = dma.recvchannel._mmio.read(dma.recvchannel._offset)
print(ctrl)
dma.recvchannel._mmio.write(dma.recvchannel._offset, (ctrl | 4) & 0xFFFFFFFE)
print(dma.recvchannel._mmio.read(dma.recvchannel._offset+0x04))
dma.recvchannel.start()
dma.sendchannel.start()

69634
1


In [ ]:
xlnk.xlnk_reset()

Lets take a look at the results of the Python HDLC code.  Notice a difference?

In [28]:
from HDLC import HDLC

class NRZI:

    def __init__(self):

        self.state = False

    def __call__(self, x):
        
        result = (x == self.state)
        self.state = x
        return result

nrzi = NRZI()
pyhdlc = HDLC()
count = 0
for b,s,l in zip(dx,sample,locked):
    if s:
        packet = pyhdlc(nrzi(b), l)
        if packet is not None:
            count += 1
            print(count, AX25(packet[1]))

1 WA6YLB-8>APRS,N6EX-11:$ULTW00000000----0000----000086A00001----0000000000000000???P
2 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!?q?
3 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!???
4 N6XQY-8>GPSLJ,RELAY,WIDE2-5:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73??B
5 WA6YLB>APRX46,WA6YLB-14,W6SCE-5:>081839z wa6ylb@theworks.com??f
6 KC6HUR-2>S4QVYV,W6SCE-5:'.4&l-/k/]"7q}???
7 N6XQY-8>GPSLJ,N6EX-9:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73???
8 KC6BLF-12>S4PWYS,N6EX-11:'-U l{(u/]"5\}Lost in the West!?;:
9 K6KMA-2>GPSLK,N6EX-3:$GPRMC,013647,A,3350.076,N,11806.996,W,028.3,180.5,231105,013.5,E*69????
10 AE6GR-14>S4PXYW,WIDE2-5:'._|l tv/]"6[}?:c
11 AE6GR-14>S4PXYW,N6EX-3:'._|l tv/]"6[}???
12 AE6MP>SS5PPQ-4,N6EX-9:`.](n->>/"4W}??
13 AE6MP>SS5PPQ-4,N6EX-3:`.](n->>/"4W}??
14 AE6MP>SS5PPQ-2,N6EX-11:`.](n->>/"4W}??
15 WA6YLB-8>S6QWSY,WA6YLB-14,N6EX-11:'/`0n>vR/]"56}???
16 K6LAR-2>APRS

The following packet is missing from the above output of the Python HDLC implementation:

```
17 KD6UZM-14>S3UWTS,WB6JAR-4,WIDE2-3:`-)?l ?v\":r} 46
```